In [1]:
import requests
import numpy as np
import pandas as pd
import os


# Función para obtener datos históricos de una criptomoneda
def obtener_datos_historicos(criptomoneda):
    url = f"https://api.binance.com/api/v3/klines?symbol={criptomoneda}USDT&interval=1d&limit=1000"
    respuesta = requests.get(url)
    datos = respuesta.json()
    
    # Extraer los datos relevantes: Fecha, Precio de cierre, Volumen, Cap. de mercado (lo aproximamos usando el precio de cierre por volumen)
    fechas = [item[0] for item in datos]
    precios_cierre = [float(item[4]) for item in datos]
    volumenes = [float(item[7]) for item in datos]
    cap_mercado = [precio * volumen for precio, volumen in zip(precios_cierre, volumenes)]
    
    df = pd.DataFrame({
        'Fecha': fechas,
        'Precio_Cierre': precios_cierre,
        'Volumen_24h': volumenes,
        'Cap_Mercado': cap_mercado
    })
    
    return df


def obtener_nombre_completo(symbol):
    url = f"https://api.binance.com/api/v3/exchangeInfo"
    respuesta = requests.get(url)
    data = respuesta.json()

    for coin_info in data['symbols']:
        if coin_info['symbol'] == f"{symbol}USDT":
            # Aquí deberías cambiar 'baseAssetName' por el campo correcto después de inspeccionar la respuesta
            return coin_info['baseAsset']  
    return None



# Función para calcular la volatilidad de 30 días
def calcular_volatilidad(precios):
    rendimientos = np.diff(np.log(precios))
    volatilidad = np.std(rendimientos) 
    return volatilidad

# Lista de criptomonedas
criptomonedas = ["BTC", "ETH", "XRP", "MATIC", "SOL", "ADA", "DOGE", "BNB", "DOT"]

# Diccionario para almacenar dataframes por criptomoneda
dataframes = {}

for moneda in criptomonedas:
    df = obtener_datos_historicos(moneda)
    
    # Calcular la volatilidad para cada fila del DataFrame y agregarla como nueva columna
    df['Volatilidad_30d'] = df['Precio_Cierre'].rolling(window=30).apply(calcular_volatilidad).round(3)

    df.fillna(0, inplace=True)

    # Convertir la columna de fecha en formato Unix a formato datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'], unit='ms')

    # Convertir la fecha al formato deseado YY/MM/DD
    df['Fecha'] = df['Fecha'].dt.strftime('%y-%m-%d')

    # Guardar el DataFrame en el diccionario
    dataframes[moneda] = df
    


for moneda, df in dataframes.items():
    # Transformar 'Volumen_24h'
    df['Volumen_24h'] = df['Volumen_24h']
    
    # Transformar 'Cap_Mercado'
    df['Cap_Mercado'] = df['Cap_Mercado']
    
        # Transformar 'Precio_Cierre'
    df['Precio_Cierre'] = df['Precio_Cierre']
    


    nombre_completo = obtener_nombre_completo(moneda)
    
    if nombre_completo:
        with open(f"{nombre_completo}.csv", "w") as file:
            # Escribir las cabeceras
            cabeceras = ["Fecha", "Precio_Cierre", "Volumen_24h", "Cap_Mercado", "Volatilidad_30d"]
            file.write(",".join(cabeceras) + "\n")

            # Escribir cada fila manualmente excepto la última
            for _, row in df[:-1].iterrows():
                valores = [str(val) for val in row]
                file.write(",".join(valores) + "\n")
            
            # Escribir la última fila sin el salto de línea al final
            valores = [str(val) for val in df.iloc[-1]]
            file.write(",".join(valores))
    
    
        """aqui haremos parte del ETL:

        Para columnas numéricas:

        count: número de valores no nulos.
        mean: media aritmética.
        std: desviación estándar.
        min: valor mínimo.
        25%: percentil 25.
        50%: mediana o percentil 50.
        75%: percentil 75.
        max: valor máximo.

        Para columnas no numéricas (por ejemplo, tipo object o categorial):

        count: número de valores no nulos.
        unique: número de valores únicos.
        top: valor más común.
        freq: frecuencia del valor más común."""

        print("-----------------------------")
        print(moneda)
        print("-----------------------------")
        print(df.info())
        print("-----------------------------")
        print(df.describe(include="all"))
    print("--------------------------------------------------------")
     
# Aquí, dataframes es un diccionario donde las claves son los nombres de las criptomonedas y 
# los valores son DataFrames con las columnas 'Fecha', 'Precio_Cierre', 'Volumen_24h', 'Cap_Mercado' y 'Volatilidad_30d'.
dataframes['SOL']



-----------------------------
BTC
-----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fecha            1000 non-null   object 
 1   Precio_Cierre    1000 non-null   float64
 2   Volumen_24h      1000 non-null   float64
 3   Cap_Mercado      1000 non-null   float64
 4   Volatilidad_30d  1000 non-null   float64
dtypes: float64(4), object(1)
memory usage: 39.2+ KB
None
-----------------------------
           Fecha  Precio_Cierre   Volumen_24h   Cap_Mercado  Volatilidad_30d
count       1000    1000.000000  1.000000e+03  1.000000e+03      1000.000000
unique      1000            NaN           NaN           NaN              NaN
top     20-11-29            NaN           NaN           NaN              NaN
freq           1            NaN           NaN           NaN              NaN
mean         NaN   34551.043140  3.1

,Fecha,Precio_Cierre,Volumen_24h,Cap_Mercado,Volatilidad_30d
0,20-11-29,1.9153,1.640192e+06,3.141461e+06,0.000
1,20-11-30,1.9665,2.153817e+06,4.235480e+06,0.000
2,20-12-01,1.9738,4.264963e+06,8.418184e+06,0.000
3,20-12-02,2.1209,4.622524e+06,9.803912e+06,0.000
4,20-12-03,2.0967,6.879027e+06,1.442326e+07,0.000
...,...,...,...,...,...
995,23-08-21,21.2000,5.588012e+07,1.184658e+09,0.031
996,23-08-22,20.5900,7.426594e+07,1.529136e+09,0.030
997,23-08-23,21.7400,7.808294e+07,1.697523e+09,0.031
998,23-08-24,21.0600,5.803070e+07,1.222127e+09,0.028


<strong>Conclusiones:

No hay datos faltantes o nulos
Noy hay datos duplicados
en todos los csv hay 1000 filas
con datos que van desde el dia 29-11-20 al dia 25-8-23
los outliers estan analizados en el archivo EDA del directorio principal
</strong>